In [ ]:
from __future__ import annotations
import datetime as dt
from typing import NamedTuple

from micro_namedtuple_sqlite_persister.persister import Engine, enable_included_adaptconverters
enable_included_adaptconverters()

In [ ]:
class MyModel(NamedTuple):
    id: int | None
    name: str
    date: dt.datetime
    score: float | None

In [ ]:
# engine = Engine(":memory:")
engine = Engine("example.db", echo_sql=False)
engine.ensure_table_created(MyModel, force_recreate=True)

In [ ]:
row = MyModel(None, "Bart", dt.datetime.now(), 6.5)
row = engine.insert(row)
engine.connection.commit()
row

In [ ]:
engine.get(MyModel,row.id)

In [ ]:
from random import random
randfloat = random()
assert row.score is not None
engine.update(row._replace(score=row.score*randfloat))
engine.connection.commit()

In [ ]:
row2 = engine.insert(MyModel(None, "foo", dt.datetime.now(), 6.5))

print(f"\nAbout to delete the recently insert row: {row2} by id")
engine.delete(MyModel, row2.id)

engine.connection.commit()

In [ ]:
row3 = engine.insert(MyModel(None, "bar", dt.datetime.now(), 9.5))

print(f"\nAbout to delete the recently insert row: {row3}, by row instance")
engine.delete(row3)

engine.connection.commit()

In [ ]:
# insert many
for i in range(25000):
    engine.insert(MyModel(None, "foo", dt.datetime.now(), random()*100))

engine.connection.commit()

In [ ]:
# update many
for id in range(1, 5001):
    engine.update(MyModel(id, "drew", dt.datetime.now(), random()*100))

engine.connection.commit()

# Querying

In [ ]:
from micro_namedtuple_sqlite_persister.query import select, gt, and_, eq

M, q = select(MyModel, where=gt(MyModel.score, 99.7))

c = 1
for r in engine.query(M,q).fetchall():
    print(f"{r.score:5.1f}", end=" ")
    if c% 30 == 0:
        print()
    c += 1
print()



In [ ]:
M, q = select(MyModel, where=gt(MyModel.name, "bart"), limit=1)
engine.query(M, q).fetchone()



In [ ]:
# Demo raw sql
class AverageScoreResults(NamedTuple):
    avg_score: float
    scorecount: int

sql = 'select avg(score),count(*) from MyModel'

result = engine.query(AverageScoreResults, sql).fetchone()
print(f'The table has {result.scorecount} rows, with and average of {result.avg_score:0.2f}')

# Save and load related models recursively

In [ ]:
class Team(NamedTuple):
    id: int | None
    name: str

class Person(NamedTuple):
    id: int | None
    name: str
    team: Team

engine.ensure_table_created(Team)
engine.ensure_table_created(Person)

team = Team(None, "Team A")
person = Person(None, "Alice", team)
person = engine.insert(person)
engine.connection.commit()
row = engine.query(Person, "SELECT * FROM Person LIMIT 1;").fetchone()

print(row)
row.team.name

## recursive saving of a giant BOM

In [ ]:
class BOM(NamedTuple):
    id: int | None
    name: str
    value: float
    child_a: BOM | None
    child_b: BOM | None

engine.ensure_table_created(BOM, force_recreate=False)

from random import random, choice
node_count = 0
def generate_node_name_node(depth: int) -> str:
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    return f"{choice(alphabet)}{choice(alphabet)}{choice(alphabet)}{depth:05d}_{node_count}"


# create a giant BOM, of 15 levels deep
def create_bom(depth: int) -> BOM:
    global node_count
    node_count += 1

    if depth == 1:
        child_a = None
        child_b = None
    else:
        child_a = create_bom(depth-1)
        child_b = create_bom(depth-1)

    return BOM(None, generate_node_name_node(depth), random()*1000 - 500, child_a, child_b)

root = create_bom(13)
print(f"Created a BOM with {node_count} nodes")

In [ ]:
inserted_root = engine.insert(root)
engine.connection.commit()

print(f"Inserted BOM with id: {inserted_root.id}")

In [ ]:
recovered_root = engine.get(BOM, inserted_root.id)

def count_nodes(node: BOM | None) -> int:
    if node is None:
        return 0
    return 1 + count_nodes(node.child_a) + count_nodes(node.child_b)

print(f"Recovered BOM with {count_nodes(recovered_root)} nodes")


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def add_nodes_edges(G: nx.Graph, node: BOM | None):
    if node is None:
        return

    G.add_node(node.id, label=node.name)
    if node.child_a is not None:
        G.add_edge(node.id, node.child_a.id)
        add_nodes_edges(G, node.child_a)

    if node.child_b is not None:
        G.add_edge(node.id, node.child_b.id)
        add_nodes_edges(G, node.child_b)

G = nx.Graph()
add_nodes_edges(G, recovered_root)

pos = nx.nx_agraph.graphviz_layout(G, prog="twopi", args="")
plt.figure(figsize=(10, 10))
def alpha(x, a=-.8, b=6.5):
    import math
    return 1 / (1 + math.exp(-a * (math.log(x) - b)))
nx.draw(G, pos, node_size=10, alpha=alpha(node_count), node_color="blue", with_labels=node_count<1200, labels=nx.get_node_attributes(G, "label"))
plt.axis("equal")
plt.show()

In [ ]:
#engine.query(*select(BOM)).fetchall() TODO: make a id select instead of Model

# Persisting Custom Types: Adapt/Convert

In [ ]:
import pandas as pd
import pickle

from micro_namedtuple_sqlite_persister.persister import register_adapt_convert

def adapt_df(obj: pd.DataFrame) -> bytes:
    return pickle.dumps(obj)


def convert_df(data: bytes) -> pd.DataFrame:
    return pickle.loads(data)


register_adapt_convert(pd.DataFrame, adapt_df, convert_df, overwrite=True)


class MyModel2(NamedTuple):
    id: int | None
    name: str
    df: pd.DataFrame

engine.ensure_table_created(MyModel2)

df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
row = engine.insert(MyModel2(None, "foo", df))

engine.get(MyModel2, row.id).df


# Error Cases

In [ ]:
# inserting a row with an id that already exists will raise an error
engine.insert(MyModel(row.id, "bar", dt.datetime.now(), 3.14))

In [ ]:
# Trying to update a row without specifying an id will raise an error
engine.update(MyModel(None, "bar", dt.datetime.now(), 3.14))

In [ ]:
# Raises an error if the id does not exist
engine.update(MyModel(878787879879, "bar", dt.datetime.now(), 3.14))

In [ ]:
# If schema already exists, but is not correct

class MyModelExists(NamedTuple): # type: ignore this is part of the error
    id: int | None
    name: str
engine.ensure_table_created(MyModelExists)

class MyModelExists(NamedTuple):
    id: int | None
    name: str | None
engine.ensure_table_created(MyModelExists)

In [ ]:
# you have to have id: `int | None` as the first field

class MyModelMissingId(NamedTuple):
    name: str

engine.ensure_table_created(MyModelMissingId)